## Problem Data

#### We can improve our code's modifiability and reusability by separating the data from the structure.

In [2]:
# An Aray in Julia is a list of elements

trophy_types = [:football, :soccer] 
# these are the possible trophy types
# this array is of type "Symbol."
# Putting a colon in front of an element makes it a "Symbol" type.
# typically Sumbol arrays are easier to implement in JuMP
# than other types (such as string, integer, etc.)

# Dictionary in Julia maps a key (any type) to an element (any type)
wood_req = Dict(:football => 4, :soccer => 2)
# how much wood each trophy type will use

# in this Dictionary, we are mapping keys of Symbol type
plaque_req = Dict(:football => 1, :soccer => 1)
profit = Dict(:football => 12, :soccer=> 9)

# we are told the amount of each resource we have available
wood_avail = 4800
plaques_avail = 1750
football_avail = 1000
soccer_avail = 1500; # semicolons suppress output

In [17]:
println(trophy_types)
println(trophy_types[1])
println(wood_req)
println(wood_req[:soccer])
println(profit[:football])

[:football, :soccer]
football
Dict(:football => 4, :soccer => 2)
2
12


## Top Brass Model

#### Now that we've defined our data we can build a general model

In [25]:
# always specify which packages you're going to use
using JuMP, Clp

# create a new model object, specifying the solver
m = Model(Clp.Optimizer)

# trophy variable object is now a Dictionary indexed over 
# trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# maximize profit by summing (profit/trophy * # trophies)
# for each type
@objective(m, Max, sum(profit[tr] * trophy[tr] for tr in trophy_types))

@constraint(m, sum(wood_req[tr] * trophy[tr] for tr in trophy_types) <= wood_avail)
@constraint(m, sum(plaque_req[tr] * trophy[tr] for tr in trophy_types) <= plaques_avail)
@constraint(m, trophy[:football] <= football_avail)
@constraint(m, trophy[:soccer] <= soccer_avail)

status = optimize!(m)

println(value.(trophy))

# note the output is quite ugly, we'll se how to make it look nicer soon.
println("Total profit will be \$", objective_value(m))
println("We will use ",  value(sum(wood_req[tr] * trophy[tr] for tr in trophy_types) ), " board feet of wood")
println("We will use ", value(sum(plaque_req[tr] * trophy[tr] for tr in trophy_types)), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


In [24]:
println(trophy)
println()
println(trophy[:football])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{VariableRef}:
 trophy[football]
 trophy[soccer]

trophy[football]


#### Another way to do the same thing uisng "expression" objects:

In [29]:
# Create expressions for the total amount of wood and plaques
# used over all trophies
# (<model name>, <expression name>, <algebraic function>)
@expression(m, tot_wood, sum(wood_req[tr] * trophy[tr] for tr in trophy_types))
@expression(m, tot_plaques, sum(plaque_req[tr] * trophy[tr] for tr in trophy_types))
# create an expression for the total amount of profit
@expression(m, tot_profit, sum(profit[tr] * trophy[tr] for tr in trophy_types))

# constraints become much simpler
@constraint(m, tot_wood <= wood_avail)
@constraint(m, tot_plaques <= plaques_avail)
@constraint(m, trophy[:football] <= football_avail)
@constraint(m, trophy[:soccer] <= soccer_avail)

# the objective is also very simple
@objective(m, Max, tot_profit)

# solve the instance of the model 
status = optimize!(m)

# print interesting solution components, including expression values
println(value.(trophy))
println("Total profit will be \$", objective_value(m))
println("We will use ", value(tot_wood), " board feet of wood")
println("We will use ", value(tot_plaques), " plaques")

LoadError: An object of name tot_wood is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :tot_wood)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:tot_wood]`.
